# Example Embeddings OpenAI

## Evaluar cantidad de tokens a procesar:

In [19]:
# Evaluar cantidad de tokens a procesar:
# !pip install tiktoken
import tiktoken
import pandas as pd

def num_tokens_from_string(text, encodig_name):
    encoding = tiktoken.get_encoding(encodig_name)
    num_tokens = len(encoding.encode(text))
    return num_tokens

In [20]:
# Cargando DataSet de prueba
df = pd.read_csv('../../Info_data/generic-food.csv')
df.head()

,FOOD NAME,SCIENTIFIC NAME,GROUP,SUB GROUP
0,Angelica,Angelica keiskei,Herbs and Spices,Herbs
1,Savoy cabbage,Brassica oleracea var. sabauda,Vegetables,Cabbages
2,Silver linden,Tilia argentea,Herbs and Spices,Herbs
3,Kiwi,Actinidia chinensis,Fruits,Tropical fruits
4,Allium (Onion),Allium,Vegetables,Onion-family vegetables


In [23]:
# Crea nueva columna con el los token que representa la columna FOOD NAME
encoding_name="cl100k_base"
df['total_tokens'] = df['FOOD NAME'].apply(lambda x : num_tokens_from_string(x,encoding_name) )
df.head()

,FOOD NAME,SCIENTIFIC NAME,GROUP,SUB GROUP,total_tokens
0,Angelica,Angelica keiskei,Herbs and Spices,Herbs,2
1,Savoy cabbage,Brassica oleracea var. sabauda,Vegetables,Cabbages,4
2,Silver linden,Tilia argentea,Herbs and Spices,Herbs,3
3,Kiwi,Actinidia chinensis,Fruits,Tropical fruits,2
4,Allium (Onion),Allium,Vegetables,Onion-family vegetables,6


In [24]:
# Total de token que se gastaria procesar todo:
sum(df['total_tokens'])

2947

## Utilizar API OpenAI

In [25]:
# !pip install openai
import openai
from getpass import getpass
import os

In [ ]:
OPENAI_API_KEY = getpass('Enter the secret value: ')
openai.api_key = OPENAI_API_KEY

In [27]:
# Generar embeddings:
def get_embedding(text, model='text-embedding-ada-002'):
    text = text.replace('\n','')
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [ ]:
df['ada_embeddings'] =  df['FOOD NAME'].apply(lambda x :get_embedding(x, model='text-embedding-ada-002') )
df.head()

In [ ]:
embedding_prueba = get_embedding('esto es una prueba de embeddings para openAI')
len(embedding_prueba)

In [ ]:
# Dataset para visualizar:
df_embeddings = pd.DataFrame(list(df['ada_embeddings']))
df_embeddings.head()

In [ ]:
# embeddings:
df_embeddings.to_csv('embedding_food.tsv',sep='\t',index=False, header=False)
# metadata:
df[['FOOD NAME','GROUP','SUB GROUP']].to_csv('labels_food.tsv',sep='\t',index=False, header=True)